# Message Passing for Story Telling

## Setup

In [2]:
import os
os.chdir('/home/zola/Projects/KBQA_archive/')

# path to KG
from subprocess import Popen, PIPE
hdt_lib_path = "/home/zola/Projects/hdt-cpp-molecules/libhdt"
hdt_file = 'data/wikidata2018_09_11.hdt'
# namespace = "http://www.wikidata.org/"

# connect to indices
from index import IndexSearch
e_index = IndexSearch('wikidata201809e')  # entity index
p_index = IndexSearch('wikidata201809p')  # predicate index

## Get Data

In [1]:
# define the seed entity 
# sample of Wikipedia articles about famous biographies
URLs = ["https://en.wikipedia.org/wiki/Pieter_Bruegel_the_Elder", "https://en.wikipedia.org/wiki/Vincent_van_Gogh",
        "https://en.wikipedia.org/wiki/Rembrandt", "https://en.wikipedia.org/wiki/Émile_Zola"]
# pick one of articles as a seed entity to be the focus of the generated story
seed_url = URLs[2]
print seed_url

# get WikiData ID
import requests, json
wiki_api_query = "https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&format=json&titles=%s"

def link_wiki_data(url):
    label = url.strip('/').split('/')[-1]
    response = requests.get(wiki_api_query%label)
    URI = json.loads(response.text)['entities'].keys()[0]
    return label, URI

seed_label, seed_URI = link_wiki_data(seed_url)
print seed_label, seed_URI

https://en.wikipedia.org/wiki/Rembrandt
Rembrandt Q5598


In [4]:
# get context subgraph: 2 hops away from the seed
seed_entity = "http://www.wikidata.org/entity/" + seed_URI
print(seed_entity)

print("Loading..")
p = Popen(["%s/tools/hops"%hdt_lib_path, "-t", "<%s>"%seed_entity, '-n', '2', hdt_file], stdin=PIPE, stdout=PIPE, stderr=PIPE, cwd=hdt_lib_path)
subgraph_str, err = p.communicate()
# size of the subgraph (M triples)
print len(subgraph_str)

http://www.wikidata.org/entity/Q5598
Loading..


KeyboardInterrupt: 

In [5]:
# extract the Wikipedia story sequence
# get the corresponding Wiki page
page = requests.get(seed_url).text
print(page)
# scrape the links
import scrape
# collect the sequence of URLs, the corresponding WikiData URIs and HDT IDs
def get_wiki_sequence(scraper):
    '''
    scraper is a function extracting links from a Wiki page
    '''
    IDs, URIs  = [], []
    links = scraper(page)
    for link in links:
        _, URI = link_wiki_data(link['url'])
        # skip entities missing from WikiData
        if URI != '-1':
            URIs.append(URI)
#             print(URI)
            # look up entity ID in index
            matches = e_index.match_entities(entity_uri, match_by='uri')
            if matches:
                IDs.append(str(matches[0]['_source']['id']))

    return IDs, URIs

# get only the links from the article summary
# summary_URIs = get_wiki_sequence(scrape.extract_summary_links)
# print("%d summary URIs extracted"%len(summary_URIs))

# get only the links from the article without the summary
story_IDs, story_URIs = get_wiki_sequence(scrape.extract_story_links)
print("%d story URIs extracted"%len(story_URIs))

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Rembrandt - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XmjRVwpAADkAAJ58seYAAADS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Rembrandt","wgTitle":"Rembrandt","wgCurRevisionId":943712572,"wgRevisionId":943712572,"wgArticleId":4254144,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with Dutch-language sources (nl)","Webarchive template wayback links","CS1 Dutch-langua

TypeError: expected string or buffer

In [ ]:
# check how many of the story entities are within the subgraph
neighbour_entities = []

# parse the subgraph
story_IDs = []
for uri in story_URIs:

print('Parsing subgraph..\n')
for triple_str in subgraph_str.strip().split('\n'):
    terms = triple_str.split()
    s, p, o = terms
    # TODO check labels match wo prefix?
    if s in story_URIs:
        neighbour_entities.append(s)
    if o in story_URIs:
        neighbour_entities.append(o)

print len(neighbour_entities)

## Message Passing

In [6]:
# graph activation by the seed entity
# look up local entity id
q_ids = entities[seed_URI]
# graph activation vector
X = np.zeros(len(entities))
X[q_ids] = 1
# only single entity is activated
assert sum(X) == 1

NameError: name 'entities' is not defined

In [2]:
# 1 hop
# ! assume we know the correct predicate sequence activation TODO vector in the number of predicates 
p_activations = np.array([1, 0])
# print p_activations.shape
# activate adjacency matrices per predicate
A1 = p_activations.T * A
# print A1.shape

# collect activations
Y1 = np.zeros(len(entities))
activations1, activations2 = [], []
for i, a_p in enumerate(A1):
    # activate current adjacency matrix via input propagation
    y_p = X*a_p
    # check if there is any signal through
    if sum(y_p) > 0:
        # add up activations
        Y1 += y_p

# check output size
assert Y1.shape[0] == len(entities)

# check activated entities
n_activated = np.nonzero(Y1)[0].shape[0]
print("%d entities activated"%n_activated)

# draw top activated entities from the distribution
if n_activated:
    Y = Y1
    n = n_activated
    top = Y1.argsort()[-n:][::-1]
    activations1 = np.asarray(re_entities)[top]

    # look up activated entities by ids
    activations1_labels = [e_index.match_entities(entity_id, match_by='id', top=1)[0]['_source']['uri'] for entity_id in activations1 ]
    topn = 5
    print(activations1_labels[:topn])
    # activation values
    print Y1[top]
    print("%d answers"%len(activations1))

    # did we hit the answer set already?
    hop1_answer = set(answer_entities_ids).issubset(set(activations1))
    print hop1_answer
    if hop1_answer:
        print("%d correct answers"%len(answer_entities_ids))
        print(answers)

NameError: name 'np' is not defined